# Current

## Create test example

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import string, random, toolz, pandas, importlib, sys, os

In [7]:
terms = [(term, 1) for term in str(random.choices(string.ascii_letters, k=10**7)).split('Z')]

In [8]:
pandas.Series(len(term) for term, _ in terms).describe()

count    193097.000000
mean        257.937223
std         255.981964
min           4.000000
25%          74.000000
50%         179.000000
75%         359.000000
max        3224.000000
dtype: float64

## Run current version

In [9]:
!git stash
!git checkout main
#importlib.reload(dimod)

No local changes to save
Already on 'main'
Your branch is up to date with 'origin/main'.


In [10]:
from dimod import BinaryPolynomial as BPold

In [11]:
BPold.__init__??

Signature: BPold.__init__(self, poly, vartype)
Docstring: Initialize self.  See help(type(self)) for accurate signature.
Source:   
    @vartype_argument('vartype')
    def __init__(self, poly, vartype):
        if isinstance(poly, abc.Mapping):
            poly = poly.items()

        # we need to aggregate the repeated terms
        self._terms = terms = {}
        for term, bias in poly:

            fsterm = asfrozenset(term)

            # when SPIN-valued, s^2 == 1, so we need to handle that case
            # in BINARY, x^2 == x
            if len(fsterm) < len(term) and vartype is Vartype.SPIN:
                new = set()
                term = tuple(term)  # make sure it has .count
                for v in fsterm:
                    if term.count(v) % 2:
                        new.add(v)
                fsterm = frozenset(new)

            if fsterm in terms:
                terms[fsterm] += bias
            else:
                terms[fsterm] = bias

        self.vartype = 

In [12]:
%%prun -s cumtime -l 20
bp = BPold(terms, 'SPIN')

         9742163 function calls (9742083 primitive calls) in 29.822 seconds

   Ordered by: cumulative time
   List reduced from 47 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   29.822   29.822 {built-in method builtins.exec}
        1    0.000    0.000   29.822   29.822 <string>:1(<module>)
        1    0.000    0.000   29.822   29.822 decorators.py:289(new_f)
        1    3.253    3.253   29.822   29.822 polynomial.py:89(__init__)
  5495096   25.363    0.000   25.363    0.000 {method 'count' of 'tuple' objects}
   193097    0.741    0.000    0.783    0.000 polynomial.py:32(asfrozenset)
  3474476    0.367    0.000    0.367    0.000 {method 'add' of 'set' objects}
   386196    0.056    0.000    0.056    0.000 {built-in method builtins.len}
   193107    0.042    0.000    0.042    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
    

In [13]:
pandas.DataFrame((len(term), bias) for term, bias in bp.items()).describe()

,0,1
count,182370.000000,182370.000000
mean,18.921994,1.058820
std,7.552707,9.036463
min,1.000000,1.000000
25%,13.000000,1.000000
50%,20.000000,1.000000
75%,25.000000,1.000000
max,42.000000,3822.000000


In [14]:
term = string.ascii_letters[:12]

In [15]:
%timeit BPold({term: 1}, 'SPIN').to_binary().to_spin() == BPold({term: 1}, 'SPIN')
BPold({term: 1}, 'SPIN').to_binary().to_spin() == BPold({term: 1}, 'SPIN')

586 ms ± 89.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


True

In [16]:
%timeit BPold({term: 1}, 'BINARY').to_spin().to_binary() == BPold({term: 1}, 'BINARY')
BPold({term: 1}, 'BINARY').to_spin().to_binary() == BPold({term: 1}, 'BINARY')

703 ms ± 3.53 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


True

In [17]:
len(BPold({term: 1}, 'SPIN').to_binary().to_spin())

4096

In [18]:
len(BPold({term: 1}, 'BINARY').to_spin().to_binary())

4096

In [19]:
prun -s cumtime -l 20 bp2 = bp.relabel_variables(dict(zip(string.ascii_letters, reversed(string.ascii_letters))), inplace=False)

         19274487 function calls (19274482 primitive calls) in 6.276 seconds

   Ordered by: cumulative time
   List reduced from 66 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    6.276    6.276 {built-in method builtins.exec}
        1    0.000    0.000    6.276    6.276 <string>:1(<module>)
      2/1    2.907    1.454    6.276    6.276 polynomial.py:232(relabel_variables)
  7266348    1.095    0.000    1.480    0.000 polynomial.py:256(<genexpr>)
   547113    0.913    0.000    1.177    0.000 _collections_abc.py:742(__iter__)
  6901612    0.385    0.000    0.385    0.000 {method 'get' of 'dict' objects}
        1    0.000    0.000    0.319    0.319 polynomial.py:177(copy)
        1    0.000    0.000    0.319    0.319 decorators.py:289(new_f)
        1    0.119    0.119    0.319    0.319 polynomial.py:89(__init__)
  1458956    0.194    0.000    0.266    0.000 polynomial.py:32(asfrozenset)
   364738   

In [20]:
prun -s cumtime -l 20 bp3 = bp2.relabel_variables(dict(zip(string.ascii_letters, reversed(string.ascii_letters))), inplace=False)

         19274484 function calls (19274479 primitive calls) in 12.020 seconds

   Ordered by: cumulative time
   List reduced from 63 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   12.020   12.020 {built-in method builtins.exec}
        1    0.000    0.000   12.019   12.019 <string>:1(<module>)
      2/1    8.767    4.383   12.019   12.019 polynomial.py:232(relabel_variables)
  7266348    1.145    0.000    1.546    0.000 polynomial.py:256(<genexpr>)
   547113    0.681    0.000    0.947    0.000 _collections_abc.py:742(__iter__)
  6901612    0.401    0.000    0.401    0.000 {method 'get' of 'dict' objects}
        1    0.000    0.000    0.314    0.314 polynomial.py:177(copy)
        1    0.000    0.000    0.314    0.314 decorators.py:289(new_f)
        1    0.111    0.111    0.314    0.314 polynomial.py:89(__init__)
   364738    0.224    0.000    0.299    0.000 polynomial.py:158(__setitem__)
  1458956 

In [21]:
bp == bp2

False

In [22]:
bp == bp3

True

In [23]:
bp2 == bp3

False

# Exit interpreter
I just can't get module reloading to work properly here ...

In [24]:
print('BYE')
exit()
raise SystemExit
print('HI')

BYE


SystemExit: 

/home/daniel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Proposed

## Create test example

In [1]:
import string, random, toolz, pandas, importlib, sys, os, cytoolz, toolz

In [2]:
import itertools as it

In [3]:
terms = [(term, 1) for term in str(random.choices(string.ascii_letters, k=10**7)).split('Z')]

In [4]:
pandas.Series(len(term) for term, _ in terms).describe()

count    192040.000000
mean        259.362430
std         257.481557
min           2.000000
25%          74.000000
50%         179.000000
75%         359.000000
max        3434.000000
dtype: float64

## Run proposed version

In [5]:
!mv polynomial-perf.ipynb{,.bak}
!git checkout binary-polynomial
!git stash pop
#!touch -c ./dimod/higherorder/polynomial.py
#importlib.reload(dimod)

M	dimod/higherorder/polynomial.py
D	polynomial-perf.ipynb
Already on 'binary-polynomial'
Your branch is ahead of 'origin/binary-polynomial' by 1 commit.
  (use "git push" to publish your local commits)
No stash entries found.


In [6]:
from dimod import BinaryPolynomial as BPnew

In [7]:
BPnew.__init__??

Signature: BPnew.__init__(self, poly, vartype)
Docstring: Initialize self.  See help(type(self)) for accurate signature.
Source:   
    @vartype_argument('vartype')
    def __init__(self, poly, vartype):
        if isinstance(poly, abc.Mapping):
            poly = poly.items()

        # we need to aggregate the repeated terms
        terms = defaultdict(int)
        for term, bias in filter(tl.second, poly):
            terms[freeze_term(term, vartype)] += bias

        self._terms = tl.valfilter(bool, terms)
        self.vartype = vartype
File:      ~/dimod/dimod/higherorder/polynomial.py
Type:      function


In [8]:
%%prun -s cumtime -l 20 
bp = BPnew(terms, 'SPIN')

         4228058 function calls (4227978 primitive calls) in 3.208 seconds

   Ordered by: cumulative time
   List reduced from 46 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    3.208    3.208 {built-in method builtins.exec}
        1    0.000    0.000    3.208    3.208 <string>:1(<module>)
        1    0.005    0.005    3.207    3.207 decorators.py:289(new_f)
        1    0.213    0.213    3.203    3.203 polynomial.py:108(__init__)
   192040    2.546    0.000    2.990    0.000 polynomial.py:40(freeze_term)
  3651735    0.415    0.000    0.415    0.000 polynomial.py:47(<genexpr>)
   192050    0.016    0.000    0.017    0.000 {built-in method builtins.isinstance}
   192041    0.013    0.000    0.013    0.000 {method 'items' of 'dict' objects}
        1    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
        1    0.000    0.000    0.000    0.000 {built-in method _abc._abc_instancecheck

In [9]:
pandas.DataFrame((len(term), bias) for term, bias in bp.items()).describe()

,0,1
count,181336.000000,181336.000000
mean,18.949006,1.059029
std,7.544061,8.981906
min,1.000000,1.000000
25%,13.000000,1.000000
50%,20.000000,1.000000
75%,25.000000,1.000000
max,40.000000,3787.000000


In [10]:
term = string.ascii_letters[:12]

In [11]:
%timeit BPnew({term: 1}, 'SPIN').to_binary().to_spin() == BPnew({term: 1}, 'SPIN')
BPnew({term: 1}, 'SPIN').to_binary().to_spin() == BPnew({term: 1}, 'SPIN')

825 ms ± 106 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


True

In [12]:
%timeit BPnew({term: 1}, 'BINARY').to_spin().to_binary() == BPnew({term: 1}, 'BINARY')
BPnew({term: 1}, 'BINARY').to_spin().to_binary() == BPnew({term: 1}, 'BINARY')

491 ms ± 5.34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


True

In [15]:
BPnew({term: 1}, 'BINARY').to_spin().to_binary()

BinaryPolynomial({frozenset({'l', 'i', 'a', 'b', 'k', 'h', 'e', 'g', 'j', 'd', 'c', 'f'}): 1.0}, 'BINARY')

In [16]:
BPnew({term: 1}, 'SPIN').to_binary().to_spin()

BinaryPolynomial({frozenset({'l', 'i', 'a', 'b', 'k', 'h', 'e', 'g', 'j', 'd', 'c', 'f'}): 1.0}, 'SPIN')

In [17]:
#len(BPnew({term: 1}, 'BINARY').to_spin() * BPnew({term: 1}, 'BINARY').to_spin())

In [18]:
prun -s cumtime -l 20 bp2 = bp.relabel_variables(dict(zip(string.ascii_letters, reversed(string.ascii_letters))), inplace=False)

         8141772 function calls in 2.503 seconds

   Ordered by: cumulative time
   List reduced from 56 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    2.503    2.503 {built-in method builtins.exec}
        1    0.003    0.003    2.503    2.503 <string>:1(<module>)
        1    0.000    0.000    2.500    2.500 polynomial.py:269(relabel_variables)
        1    1.350    1.350    2.256    2.256 polynomial.py:295(<dictcomp>)
  3617473    0.547    0.000    0.737    0.000 polynomial.py:296(<genexpr>)
  3436141    0.190    0.000    0.190    0.000 {method 'get' of 'dict' objects}
   181337    0.060    0.000    0.169    0.000 _collections_abc.py:742(__iter__)
        1    0.004    0.004    0.149    0.149 decorators.py:289(new_f)
        1    0.117    0.117    0.145    0.145 polynomial.py:108(__init__)
   181336    0.073    0.000    0.109    0.000 polynomial.py:153(__getitem__)
        1    0.009    0.009    0

In [19]:
prun -s cumtime -l 20 bp3 = bp2.relabel_variables(dict(zip(string.ascii_letters, reversed(string.ascii_letters))), inplace=False)

         8141772 function calls in 2.381 seconds

   Ordered by: cumulative time
   List reduced from 56 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    2.381    2.381 {built-in method builtins.exec}
        1    0.003    0.003    2.381    2.381 <string>:1(<module>)
        1    0.000    0.000    2.378    2.378 polynomial.py:269(relabel_variables)
        1    1.218    1.218    2.131    2.131 polynomial.py:295(<dictcomp>)
  3617473    0.547    0.000    0.739    0.000 polynomial.py:296(<genexpr>)
  3436141    0.192    0.000    0.192    0.000 {method 'get' of 'dict' objects}
   181337    0.063    0.000    0.174    0.000 _collections_abc.py:742(__iter__)
        1    0.004    0.004    0.150    0.150 decorators.py:289(new_f)
        1    0.117    0.117    0.145    0.145 polynomial.py:108(__init__)
   181336    0.076    0.000    0.112    0.000 polynomial.py:153(__getitem__)
        1    0.009    0.009    0

In [20]:
bp == bp2

False

In [21]:
bp == bp3

True

In [22]:
bp2 == bp3

False